First part of laboratories were conducted by <a href="https://github.com/makskliczkowski">Maksymilian Kliczkowski</a>. He created notebooks we used in laboratories and he help me and my friends with understanding basic and advanced topics in machine learning. I am really glad that I had him as a tutor.

In first course of ML/AI we were build ML models from the ground using math.

In this notebook I will present only some task we had as a homework from notebook about scikit-learn

# Logistic regression - part taken from older tasks

#### k) Create the logistic regression algorithm using the information from above. Implement Newton-Rhapson for it as well.

In [ ]:
import scipy as sc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

## OLD CLASSES

In [ ]:
class Perceptron:
    # Initialize the bias and the dimension of the perceptron.
    def __init__(self, W : np.array,  b = 0):
        self.W = W
        self.N = len(W)
        self.b = b

    ############################## GETTERS ##############################

    def get_N(self):
        return self.N

    def get_b(self):
        return self.b

    def get_W(self):
        return self.W

    ############################## SETTERS ##############################

    # Create setters for the perceptron
    def set_N(self, N):
        self.N = N
        self.W = np.zeros(N)

    def set_b(self, b):
        self.b = b

    def set_W(self, W : np.array):
        if W.ndim != 1:
            print("Cannot set such weights -> dimension wrong")
            return
        self.N = W.shape[0]
        self.W = W

    ############################## GETTERS OVERRIDE ##############################

    def __getitem__(self, key):
        return self.W[key]

    def __setitem__(self, key, value):
        self.W[key] = value

    def __getslice(self, i, j):
        return self.W[i:j]

    # set the string output of the perceptron
    def __str__(self):
        return f"Am a perceptron of N={self.N} dimension{'s' if self.N > 1 else ''} biased with b={self.b}"

    ############################## OPERATORS OVERRIDE ##############################

    def __mul__(self, other):
        return self.activation_function(other)

    def __rmul__(self, other):
        return self.__mul__(other)

    ############################## PERCEPTRON METHODS ##############################

    '''
    Net output is the basic body action of the perceptron. On top of it, the activation function is used.
    '''
    def net_output(self, X):
        return np.dot(X, self.W) + self.b

    '''
    Here the activation function is step-like
    '''
    def activation_function(self, X):
        return np.where(self.net_output(X) >= 0.0, 1.0, 0.0).reshape((len(X),1))

    '''
    Predicts the output of perceptron -> here the class is given
    '''
    def predict(self, X):
        return self.activation_function(X)

In [ ]:
class PerceptronLinear(Perceptron):
    # Initialize the bias and the dimension of the perceptron -> but not only that
    def __init__(self, W : np.array,  b = 0, epo = 100, lr = 0.01):
        super().__init__(W, b)
        # how many learning iterations we give
        self.epo = epo
        # what is our step in the gradient
        self.lr = lr

    ############################## GETTERS ##############################

    def get_lr(self):
        return self.lr

    def get_epo(self):
        return self.epo

    ############################## SETTERS ##############################

    def set_lr(self, lr):
        self.lr = lr

    def set_epo(self, epo):
        self.epo = epo

    ######################## PERCEPTRON METHODS #########################
    def activation_function(self, X):
        '''
        As you can see the activation function has been changed to linear regression. What does reshape do?
        After you figure it out, note that it is the usual way of having the output of the machine learning algorithm. Why?
        '''
        return (self.net_output(X)).reshape(-1,1)

    def loss(self, y_true : np.array, y_pred : np.array):
        '''
        Loss function for the perceptron -> here we use the Mean Square Error
        '''
        square = np.square(y_true - y_pred)
        return np.mean(square)

    def gradient(self, x_true, y_true, prediction):
        '''
        Single step of the gradient, here it is calculated analytically (linear regression)
        '''
        delta_i = (y_true.flatten() - prediction)
        suma_w  = np.multiply(delta_i[:, np.newaxis], x_true)
        suma_b  = delta_i
        return suma_b, suma_w

    def fit(self, X, y, randomstate = None, batch = 1, verbose = False):
        '''
        Fit function allows to obtain the (probably most?) correct weights for the perceptron via the gradient descent algorithm.
        '''

        if type(X) != np.ndarray:
            X = np.array(X)
        if type(y) != np.ndarray:
            y = np.array(y).reshape(-1,1)

        # give fit the parameter randomstate and whenever it is not None, the weights
        # are reset to be random normal - this ensures random starting point of gradient descent
        if randomstate is not None:
            self.W = np.random.normal(0.0, 0.1, self.N)
            self.b = np.random.normal(0.0, 1.0)

        # Save the history of the losses. Why?
        history = []
        # If we want to calculate the gradient in buckets (look for description of the batch)
        bucket_num = len(X) // batch
        # slice the data onto batches without shuffling (no stochasticity)
        slicing = lambda x, b: x[(b-1)*batch:b*batch]

        # iterate epochs
        for epo in range(self.epo):
            # iterate batches
            loss = 0.0
            for bin in range(1, bucket_num + 1):
                X_slice = slicing(X,bin)
                y_slice = slicing(y,bin)
                # predict the output for a given slice (what is the shape of the output?)
                pred = self.predict(X_slice)

                suma_b, suma_w = self.gradient(X_slice, y_slice, pred.flatten())

                # calculate loss
                loss += self.loss(y_slice, pred.flatten())

                # update the weights
                self.W += np.mean(suma_w, axis = 0) * self.lr
                self.b += np.mean(suma_b, axis = 0) * self.lr
            # calculate average loss
            loss/=bucket_num
            if verbose:
                print(f'epo:{epo}->loss={loss}')
            history.append(loss.flatten())
        return np.array(history).flatten()

    def plot_history(self, history, ax = None):
        '''
        Basic history plot
        '''
        if ax is None:
            fig, ax = plt.subplots()
        ax.set_xlabel('epo')
        ax.set_ylabel('loss')
        ax.plot(history)

In [ ]:
class PerceptronBinary(PerceptronLinear):
    def __init__(self, W : np.array,  b = 0, epo = 100, lr = 0.01):
        super().__init__(W, b, epo, lr)

    def activation_function(self, X):
        '''
        $\Phi(x) = sign(x)
        '''
        return np.where(self.net_output(X) >= 0.0, 1.0, -1.0).reshape(-1,1)

    def loss(self, y_true : np.array, y_pred : np.array):
        '''
        Loss function for the perceptron -> here we use knowledge that the classes can be either {-1, 1}
        '''
        square = np.square(1.0-y_true * y_pred)
        return np.mean(square)

    def gradient(self, x_true, y_true, prediction):
        '''
        Single step of the gradient, here it is calculatable analytically (linear regression)
        '''
        val = np.multiply(y_true, (1.0-y_true * prediction))
        suma_w = np.multiply(val, x_true)
        suma_b = val
        return suma_b, suma_w

In [ ]:
class PerceptronLogistic(PerceptronBinary):
    def __init__(self, W : np.array,  b = 0, epo = 100, lr = 0.01):
        super().__init__(W, b, epo, lr)

    def activation_function(self, X):
        return sc.special.expit(X*self.W).shape(-1,1)

    def loss(self, y_true : np.array, y_pred : np.array):
        lg = np.log(1 - y_pred)
        mul = np.multiply((1-y_true), lg)
        lh = np.multiply(y_true, np.log(y_pred))
        return lh + mul

    def gradient(self, x_true, y_true, prediction):
      sig = self.activation_function(np.multiply(self.W * x_true))
      return np.multiply((y_true - sig), x_true)

In [ ]:
# data
X = (np.concatenate((np.random.uniform(low=-2.0, high=0.4, size=50), np.random.uniform(low=-0.4, high=3.0, size=50)))).reshape(-1,1)
Y = np.array([0]*50 + [1]*50).reshape(-1,1)

# fit the perceptron
p=PerceptronLogistic([1],0.1,500,0.01)
history=p.fit(X, Y)
p.plot_history(history)
Y_pred = p.predict(X)
plt.show()

# plotting data and prediction
plt.scatter(X,Y_pred)
plt.scatter(X,Y,c='red')
plt.xlabel('x')
plt.ylabel('class')
plt.show()

### a) Use the data below to test the final logistic regression. Do you see the probability corespondence?

In [ ]:
# data (create the X variables from -2.0 to 0.4 randomly) and concatenating it with slightly different data, make this nonseparable linearly by creating an overlap
X = (np.concatenate((np.random.uniform(low=-2.0, high=0.4, size=50), np.random.uniform(low=-0.4, high=3.0, size=50)))).reshape(-1,1)
Y = np.array([0]*50 + [1]*50).reshape(-1,1)

In [ ]:
# fit the perceptron
p       = PerceptronLogistic(np.array(np.random.random(1)))
history = p.fit(X, Y)
p.plot_history(history)
Y_pred  = p.activation_function(X)
plt.show()

# plotting data and prediction
plt.scatter(X,Y_pred)
plt.scatter(X,Y,c='red')
plt.xlabel('x')
plt.ylabel('class')
plt.show()

#### b) Implement a correct Hessian using the templates from the previous exercises.

In [ ]:
class PerceptronLogistic_v2(PerceptronBinary):
    def __init__(self, W : np.array,  b = 0, epo = 100, lr = 0.01):
        super().__init__(W, b, epo, lr)

    def activation_function(self, X):
        '''
        Sigmoid - plot it if you want to
        '''
        return (1.0/(1.0+np.exp(-self.net_output(X)))).reshape(-1,1)

    def loss(self, y_true : np.array, y_pred : np.array):
        '''
        Maximal likelyhood loss
        '''
        return -np.sum(np.multiply(y_true, np.log(y_pred)) + np.multiply((1.0-y_true), np.log(1.0-y_pred)))

    def gradient(self, x_true, y_true, prediction):
        '''
        Is of the same character as linear regression
        '''
        delta_i = (y_true.flatten() - prediction)
        suma_w  = np.multiply(delta_i[:, np.newaxis], x_true)
        suma_b  = delta_i
        return suma_b, suma_w

    def hessian(self, x_true, y_true, y_pred):
        '''
        Calculates the hessian using logistic regression sigmoid activation
        - x_true : input of the model (for convinience in flattened version)
        - y_true : supervised learning true classes
        - y_pred : predictions of the model for a given batch
        '''
        hess = np.zeros((len(x_true)+1, len(x_true)+1))

        print(x_true, y_true, y_pred)
        # we add 1 because of the bias (go through rows)
        for i in range(len(x_true) + 1):
            # taking bias into account
            for j in range(len(x_true) + 1):
                x1 = 1 if i == 0 else x_true[i-1]
                x2 = 1 if j == 0 else x_true[j-1]
                hess[i][j] = np.multiply(y_pred, (1 - y_pred)) * x1 * x2
        return hess

    def newton_rap(self, X, y, randomstate = None, verbose = False):
        '''
        Newton rap : https://www.youtube.com/watch?v=8yis7GzlXNM
        '''
        if type(X) != np.ndarray:
            X = np.array(X)
        if type(y) != np.ndarray:
            y = np.array(y).reshape(-1,1)

        # give fit the parameter randomstate and whenever it is not None, the weights
        # are reset to be random normal - this ensures random starting point of gradient descent
        if randomstate is not None:
            self.W = np.random.normal(0.0, 0.1, self.N)
            self.b = np.random.normal(0.0, 1.0)

        # Save the history of the losses. Why?
        history = []

        # iterate epochs
        for epo in range(self.epo):
            # iterate batches
            loss = 0.0
            for i, X_slice in enumerate(X):
                y_slice = y[i]
                # predict the output for a given slice (what is the shape of the output?)
                pred    = self.predict(X_slice)
                suma_b, suma_w = self.gradient(X_slice, y_slice, pred.flatten())
                # calculate the gradient
                grad    = np.array(list(suma_b.flatten()) + list(suma_w.flatten()))
                # calculate the hessian
                hessian = self.hessian(X_slice, y_slice, pred.flatten())
                # calculate the update vector (use pseudoinverse to be numerically safe)
                update  = np.linalg.pinv(hessian).dot(grad)
                # update weights
                self.W  += np.array(self.lr*update[1:])
                self.b  += np.array(self.lr*update[0])
                # calculate loss
                loss    += self.loss(y_slice, pred.flatten())
            if verbose:
                print(f'epo:{epo}->loss={loss}')
            history.append(loss.flatten())
        return np.array(history).flatten()

In [ ]:
# data
X   =   (np.concatenate((np.random.uniform(low=-2.0, high=0.4, size=50), np.random.uniform(low=-0.4, high=3.0, size=50)))).reshape(-1,1)
Y   =   np.array([0]*50 + [1]*50).reshape(-1,1)
# fit the perceptron
p       =   PerceptronLogistic_v2([1], 0.1, 400, lr = 2e-4)
history =   p.newton_rap(X, Y)
p.plot_history(history)
Y_pred  =   p.predict(X)
plt.show()

# plotting data and prediction
plt.scatter(X,Y_pred)
plt.scatter(X,Y,c='red')
plt.axhline(0.5)
plt.axvline(np.min(X[Y==1]))
plt.axvline(np.max(X[Y==0]))
plt.xlabel('x')
plt.ylabel('class')
plt.show()